In [1]:
import findspark
import os
findspark.init()


memory = '4g'
os.environ["PYSPARK_SUBMIT_ARGS"] = ' --driver-memory ' + memory + ' pyspark-shell'

import pyspark
import random

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [2]:
spark

In [3]:
from pyspark.sql.functions import explode, first
import pyspark.sql.functions as f
from pyspark.sql.functions import col, expr, when
from pyspark.sql.types import DateType

In [4]:
BASE_DIR: str = os.path.join("output", "ifood")
DATASET_DIR: str = os.path.join(BASE_DIR, "dataset")
os.path.join(DATASET_DIR, 'window_filter', "accounts_indices_for_interactions_data.csv")

'output/ifood/dataset/window_filter/accounts_indices_for_interactions_data.csv'

In [5]:
df = spark.read.parquet('../../dataset/ufg_training_data')
df_test = spark.read.parquet('../../dataset/ufg_test_data')

In [6]:
df_test.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- search_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merc_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- merc_score: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- delivery_fee: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- delivery_time: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- distance: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- original_ranking_size: integer (nullable = true)
 |-- week_day: string (nullable = true)
 |-- shift: string (nullable = true)
 |-- order_date_local: date (nullable = true)



In [7]:
df_test.count()

3764773

In [6]:
#df_sample = df.limit(5).cache()
df.show(5)

+-----+--------------------+--------------------+------+--------------------+----+---------------+
|shift|          account_id|         merchant_id|visits|        visit_events|buys|purchase_events|
+-----+--------------------+--------------------+------+--------------------+----+---------------+
| Dawn|00001c30-68c0-489...|01bc7956-e370-4cb...|     1|        [2019-03-30]|null|           null|
| Dawn|00001ec2-8a94-43c...|66948144-76f8-493...|     1|        [2019-03-31]|null|           null|
| Dawn|000033a9-c4f4-404...|9a657957-af83-41c...|     1|        [2019-06-03]|null|           null|
| Dawn|00006865-47ca-467...|4e48a8a3-b851-420...|     1|        [2019-06-03]|null|           null|
| Dawn|00008681-46b5-448...|8c69dce9-e1c4-48b...|     3|[2019-06-15, 2019...|   1|   [2019-06-15]|
+-----+--------------------+--------------------+------+--------------------+----+---------------+
only showing top 5 rows



In [7]:
df_test.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- search_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merc_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- merc_score: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- delivery_fee: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- delivery_time: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- distance: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- original_ranking_size: integer (nullable = true)
 |-- week_day: string (nullable = true)
 |-- shift: string (nullable = true)
 |-- order_date_local: date (nullable = true)



In [8]:
df_test.select(['order_id', 'account_id', 'merchant_id', 'merc_list', 'order_date_local']).show(1)

+--------------------+--------------------+--------------------+--------------------+----------------+
|            order_id|          account_id|         merchant_id|           merc_list|order_date_local|
+--------------------+--------------------+--------------------+--------------------+----------------+
|f2f028cb-5e00-43b...|11090744-4b7d-444...|7b126ecc-ec86-45d...|[fe2f0c1b-d024-4b...|      2019-06-19|
+--------------------+--------------------+--------------------+--------------------+----------------+
only showing top 1 row



In [11]:
df_test.select(f.min('order_date_local')).show()

+---------------------+
|min(order_date_local)|
+---------------------+
|           2019-06-17|
+---------------------+



In [12]:
df_test.select(f.max('order_date_local')).show()

+---------------------+
|max(order_date_local)|
+---------------------+
|           2019-06-24|
+---------------------+



### Explode 

In [11]:


df_t = df.select(df.account_id, df.merchant_id, 
                 explode(df.visit_events).alias('visit_entent'),
                 f.col('purchase_events')[0].alias('purchase_event'))
df_t = df_t.withColumn("visit_entent",df_t['visit_entent'].cast(DateType()))
df_t = df_t.withColumn('buys', when(df_t.purchase_event == df_t.visit_entent, 1).otherwise(0)).persist( pyspark.StorageLevel.MEMORY_AND_DISK_2 )

df_t.show()

+--------------------+--------------------+------------+--------------+----+
|          account_id|         merchant_id|visit_entent|purchase_event|buys|
+--------------------+--------------------+------------+--------------+----+
|00001c30-68c0-489...|01bc7956-e370-4cb...|  2019-03-30|          null|   0|
|00001ec2-8a94-43c...|66948144-76f8-493...|  2019-03-31|          null|   0|
|000033a9-c4f4-404...|9a657957-af83-41c...|  2019-06-03|          null|   0|
|00006865-47ca-467...|4e48a8a3-b851-420...|  2019-06-03|          null|   0|
|00008681-46b5-448...|8c69dce9-e1c4-48b...|  2019-06-15|    2019-06-15|   1|
|00008681-46b5-448...|8c69dce9-e1c4-48b...|  2019-06-17|    2019-06-15|   0|
|00008681-46b5-448...|8c69dce9-e1c4-48b...|  2019-03-31|    2019-06-15|   0|
|0000a894-0e1d-4fb...|abff5b83-6a67-461...|  2019-05-30|          null|   0|
|0000bba0-80b8-4e6...|11266bdb-617c-4ed...|  2019-03-27|    2019-03-27|   1|
|0000bba0-80b8-4e6...|11266bdb-617c-4ed...|  2019-03-27|    2019-03-27|   1|

In [12]:
df_t.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- visit_entent: date (nullable = true)
 |-- purchase_event: string (nullable = true)
 |-- buys: integer (nullable = false)



In [13]:
df_t.select(f.min('visit_entent')).show()

+-----------------+
|min(visit_entent)|
+-----------------+
|       2019-03-26|
+-----------------+



In [14]:
df_t.select(f.max('visit_entent')).show()

+-----------------+
|max(visit_entent)|
+-----------------+
|       2019-06-17|
+-----------------+



In [15]:
#df_t.select(f.max('visit_entent_date')).show()

In [16]:
#df_t.persist( pyspark.StorageLevel.MEMORY_AND_DISK_2 )

In [17]:
df_t.write.parquet('/media/marlesson/Backup/dataset/ifood_interactions')

### Filter

In [18]:
df_t.show(2)

+--------------------+--------------------+------------+--------------+----+
|          account_id|         merchant_id|visit_entent|purchase_event|buys|
+--------------------+--------------------+------------+--------------+----+
|00001c30-68c0-489...|01bc7956-e370-4cb...|  2019-03-30|          null|   0|
|00001ec2-8a94-43c...|66948144-76f8-493...|  2019-03-31|          null|   0|
+--------------------+--------------------+------------+--------------+----+
only showing top 2 rows



In [19]:
df_1w = df_t.filter(f.col("visit_entent") > f.lit("2019-06-10")).cache()
df_1w.count()

28272839

In [20]:
df_1m = df_t.filter(f.col("visit_entent") > f.lit("2019-05-17")).cache()
df_1m.count()

91571810

In [21]:
# all        - 127.407.924
# 2019-06-10 - 28.272.839
# 2019-05-17 - 91.571.810

### Implode

In [22]:
#df_f.show()

In [23]:
#df.show(5)

In [24]:
df_1w_t = df_1w.groupby([df_1w.account_id, df_1w.merchant_id]).agg(
            f.count('visit_entent').alias('visits'),
            f.collect_list('visit_entent').alias('visit_entents'),
            when(f.sum('buys') >= 1, 1).alias('buys'),
            when(f.size(f.collect_set('purchase_event')) == 0, f.lit(None)).otherwise(f.collect_set('purchase_event')).alias('purchase_events')
        ).cache()

df_1w_t.show()

+--------------------+--------------------+------+--------------------+----+---------------+
|          account_id|         merchant_id|visits|       visit_entents|buys|purchase_events|
+--------------------+--------------------+------+--------------------+----+---------------+
|00002a9b-c04b-487...|61540dea-fea3-4a4...|     1|        [2019-06-13]|null|           null|
|00002a9b-c04b-487...|f5068902-033e-425...|     1|        [2019-06-13]|null|   [2019-05-11]|
|00006d0a-8a5e-40a...|25b28f17-bcf2-47a...|     1|        [2019-06-17]|null|           null|
|000135d6-78f9-409...|58d4f0c8-4a4e-4ef...|     2|[2019-06-16, 2019...|null|           null|
|00027048-30c0-490...|54c9b5dd-39ac-410...|     1|        [2019-06-16]|null|           null|
|00029e81-e550-47d...|2722a841-fcf4-4ae...|     1|        [2019-06-12]|null|   [2019-06-06]|
|0003749a-d3a4-425...|731f5850-fc71-47e...|     1|        [2019-06-14]|null|           null|
|00046739-a44e-46d...|3eadb121-25d4-4b3...|     3|[2019-06-16, 2019...

In [25]:
df_1w_t.write.parquet('/media/marlesson/Backup/dataset/ifood_training_1w')

In [26]:
df_1m_t = df_1w.groupby([df_1m.account_id, df_1m.merchant_id]).agg(
            f.count('visit_entent').alias('visits'),
            f.collect_list('visit_entent').alias('visit_entents'),
            when(f.sum('buys') >= 1, 1).alias('buys'),
            when(f.size(f.collect_set('purchase_event')) == 0, f.lit(None)).otherwise(f.collect_set('purchase_event')).alias('purchase_events')
        ).cache()

df_1m_t.show()

+--------------------+--------------------+------+--------------------+----+---------------+
|          account_id|         merchant_id|visits|       visit_entents|buys|purchase_events|
+--------------------+--------------------+------+--------------------+----+---------------+
|00002a9b-c04b-487...|61540dea-fea3-4a4...|     1|        [2019-06-13]|null|           null|
|00002a9b-c04b-487...|f5068902-033e-425...|     1|        [2019-06-13]|null|   [2019-05-11]|
|00006d0a-8a5e-40a...|25b28f17-bcf2-47a...|     1|        [2019-06-17]|null|           null|
|000135d6-78f9-409...|58d4f0c8-4a4e-4ef...|     2|[2019-06-16, 2019...|null|           null|
|00027048-30c0-490...|54c9b5dd-39ac-410...|     1|        [2019-06-16]|null|           null|
|00029e81-e550-47d...|2722a841-fcf4-4ae...|     1|        [2019-06-12]|null|   [2019-06-06]|
|0003749a-d3a4-425...|731f5850-fc71-47e...|     1|        [2019-06-14]|null|           null|
|00046739-a44e-46d...|3eadb121-25d4-4b3...|     3|[2019-06-16, 2019...

In [27]:
df_1m_t.write.parquet('/media/marlesson/Backup/dataset/ifood_training_1m')